## Prepare the latest sales record
6 months sales record history to reset the inventory aging time.
The guilty inventory count starts from the month of no sales.

1.  Delete the blank rows of the excel to have the header as 1st row.
2.  Save the excel by csv

In [2]:
import pandas as pd
import sqlite3


In [12]:
# Connect to sqlite3
conn = sqlite3.connect('guilty_inventory.sqlite3')
c = conn.cursor()

# Create tables
# t_sales table
c.execute('''
create table if not exists t_sales (sales_month text, sales_model text,
sales_qty int, sales_amt float, primary key(sales_month, sales_model))
''')

# t_purchase table
c.execute('''
create table if not exists t_purchase (purchase_month text, purchase_model text,
purchase_qty int, purchase_amt float, primary key(purchase_month, purchase_model))
''')

# t_inv table
c.execute('''
create table if not exists t_inv (inv_month text, inv_model text, inv_cat int,
inv_qty int, inv_amt float, primary key(inv_month, inv_model))
''')

# Close connection to sqlite3
c.close()
conn.close()


In [40]:
# # Sales record preparation

# targetFile = 'sales-201909-202008.csv'
# useCols = ['Billing doc. date', 'Material', 'Net Qty', 'Net $']
# dataType = {'Net Qty': int, 'Net $': float}

# df = pd.read_csv(targetFile, usecols=useCols, parse_dates=[0])
# df = df.rename(columns={'Billing doc. date': 'sales_month', 
#                         'Material': 'sales_model', 'Net Qty': 'sales_qty',
#                        'Net $': 'sales_amt'})
# # If YYYYMM string is necessary instead of datetim364[ns] enable below.
# df['sales_month'] = df['sales_month'].dt.strftime('%Y%m')
# grouped = df.groupby(['sales_month', 'sales_model'], as_index=False)
# #print(grouped.sum())
# grouped.dtypes

In [17]:
def t_sales_update_csv(targetFile):
    # Sales record preparation from csv and add to sqlite3 t_sales table
    # Copy the original sheet to a new sheet & paste value and save as csv.
    # Billing doc. date must be formatted to date (ex: 2020/8/2)

    useCols = ['Billing doc. date', 'Material', 'Net Qty', 'Net $']

    df = pd.read_csv(targetFile, usecols=useCols, parse_dates=[0])
    df = df.rename(columns={'Billing doc. date': 'sales_month', 
                        'Material': 'sales_model', 'Net Qty': 'sales_qty',
                       'Net $': 'sales_amt'})
    # If YYYYMM string is necessary instead of datetim364[ns] enable below.
    df['sales_month'] = df['sales_month'].dt.strftime('%Y%m')
    grouped = df.groupby(['sales_month', 'sales_model'], as_index=False)
    # print(grouped.sum())
    
    # Connect to sqlite3
    conn = sqlite3.connect('guilty_inventory.sqlite3')
    c = conn.cursor()
    
    # Add grouped.sum() into t_sales table
    grouped.sum().to_sql('t_sales', conn, if_exists='append', index=None)
    
    # Close connection to sqlite3
    c.close()
    conn.close()

    
t_sales_update_csv('201910.csv')

In [14]:
def t_sales_update_excel(targetFile):
    # Sales record preparation from xlsx and add to sqlite3 t_sales table
    # Copy the original sheet to a new sheet & paste value and save as xlsx.
    # Billing doc. date must be formatted to date (ex: 2020/8/2)

    useCols = ['Billing doc. date', 'Material', 'Net Qty', 'Net $']

    df = pd.read_excel(targetFile, usecols=useCols, parse_dates=[0])
    df = df.rename(columns={'Billing doc. date': 'sales_month', 
                        'Material': 'sales_model', 'Net Qty': 'sales_qty',
                       'Net $': 'sales_amt'})
    # If YYYYMM string is necessary instead of datetim364[ns] enable below.
    df['sales_month'] = df['sales_month'].dt.strftime('%Y%m')
    grouped = df.groupby(['sales_month', 'sales_model'], as_index=False)
    print(grouped.sum())

    
t_sales_update_excel('201909.xlsx')


    sales_month  sales_model  sales_qty  sales_amt
0        201909      AL100DR         84  102527.86
1        201909    AL100RD/U        408   52412.05
2        201909    AL100RT/U         30    4598.25
3        201909     AL100TDN         55   64295.78
4        201909   AL204RTC/U       1320  292547.80
..          ...          ...        ...        ...
219      201909       R21LTF         24  162714.85
220      201909       R22GTF          7   88708.00
221      201909       SFSC11          1     696.14
222      201909  SFV19.50KWP          1  570000.00
223      201909       SR560M          2   27000.00

[224 rows x 4 columns]
